<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model = MODEL_ID,
    contents = "Quando é a próxima imersão IA com Google Gemini da Alura?"
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

In [8]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config = {"tools": [{"google_search":{}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá de 12 a 16 de maio de 2025. As inscrições para o curso estão abertas até o dia 11 de maio de 2025. O curso é gratuito e online, e os participantes receberão um certificado Alura + Google.


In [10]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA Google Gemini datas', 'Alura Imersão IA com Google Gemini próxima edição']
Páginas utilizadas na resposta: convergenciadigital.com.br, thallesbenicio.com.br, alura.com.br



In [11]:
# Instalar Framework ADK de agentes do Google ################################################
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [12]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [13]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [14]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [32]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pésquisa. E sua tarefa é usar a ferramenta do Google (google_search)
      para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque no máximo de 5 lançamentos relevantes, com base na qualidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que não seja tão relevante assim
      e pode ser substituído por outro que tenha mais.
      Esses lançamentos relevantes devem ser atuais de no máximo um mês antes da data de hoje.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [33]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejado de conteúdo, especialista em redes sociais.
        Com base nos lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano
        sobre quais são os pontos mais relevantes que poderíamos abordar em um
        post em cada um deles. Você também pode usar o (google_search) para
        encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base
        nas suas pesquisas e retornar esse tema, seus pontos mais relevantes e
        plano com os assuntos a serem abordados no post que será escrito
        posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [34]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [37]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [36]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você não digitou um tópico. Por favor, tente novamente.")
else :
  print(f"Maravilha, vamos criar seu post sobre novidades {topico}" )

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n----------------- RESULTADO DO AGENTE BUSCADOR -----------------\n")
  display(to_markdown(lancamentos_buscados))
  print("----------------------------------------------------------------")

  plano_de_post = agente_buscador (topico, lancamentos_buscados)
  print("\n----------------- RESULTADO DO AGENTE PLANEJADOR -----------------\n")
  display(to_markdown(plano_de_post))
  print("----------------------------------------------------------------")

  redator_de_post = agente_buscador (topico, plano_de_post)
  print("\n----------------- RESULTADO DO AGENTE REDATOR -----------------\n")
  display(to_markdown(redator_de_post))
  print("----------------------------------------------------------------")

  revisor_de_post = agente_buscador (topico, redator_de_post)
  print("\n----------------- RESULTADO DO AGENTE REVISOR -----------------\n")
  display(to_markdown(revisor_de_post))
  print("----------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA
Maravilha, vamos criar seu post sobre novidades Agentes de IA

----------------- RESULTADO DO AGENTE BUSCADOR -----------------



> Para fornecer as últimas notícias sobre lançamentos relevantes de agentes de IA, farei algumas pesquisas no Google. Aqui estão as consultas que usarei:
> 
> 
> Com certeza! Aqui estão alguns dos lançamentos e tendências mais relevantes no mundo dos agentes de IA, com foco nos últimos meses:
> 
> **1. Adoção e Impacto no Comércio:**
> 
> *   Agentes de IA estão se tornando onipresentes e devem revolucionar a forma como interagimos com as finanças e o comércio. Um relatório da Visa indica que 82% dos brasileiros demonstram interesse em assistentes pessoais com IA.
> *   Empresas de todos os setores estão se adaptando para aproveitar a transformação digital impulsionada pela IA, buscando autonomia, eficiência e redução de custos.
> *   Em 2025, espera-se que a IA deixe de ser uma promessa e se torne uma peça-chave na economia, com empresas investindo fortemente em sistemas mais personalizados e eficientes.
> 
> **2. Avanços Tecnológicos e Novas Ferramentas:**
> 
> *   Grandes empresas como IBM, Anthropic, Amazon, Google e OpenAI estão anunciando atualizações e lançamentos que prometem revolucionar o cenário da IA, com ferramentas inovadoras e integração avançada entre agentes.
> *   A IBM anunciou atualizações no watsonx Orchestrate, oferecendo soluções para integrar, customizar e implantar agentes, além de um Agent Catalog que simplifica a orquestração de projetos complexos.
> *   Anthropic introduziu a funcionalidade de busca na web em sua API, permitindo que o agente Claude forneça respostas mais precisas e contextualizadas.
> *   A Microsoft revelou novos agentes para o Security Copilot e proteções para IA. Novos agentes de IA do Copilot criam relatórios e analisam dados em minutos.
> 
> **3. Agentes de IA como Criptomoedas:**
> 
> *   A interseção entre criptomoedas e inteligência artificial está se tornando uma área de crescente interesse, com inovações em IA catalisando a evolução de criptomoedas.
> *   Criptomoedas de IA utilizam inteligência artificial para automatizar tarefas, simplificar operações e oferecer análises preditivas para traders.
> *   Exemplos de criptomoedas de IA incluem Mind of Pepe (MIND), SUBBD e Fantasy Pepe (FEPE).
> 
> **4. Desafios e Expectativas:**
> 
> *   Especialistas da Andreessen Horowitz (a16z) apontam que o termo "agente de IA" está se desgastando, com muitas pessoas usando a expressão sem entender o conceito.
> *   Para que a IA substitua trabalhadores humanos, é preciso resolver questões técnicas como memória de longo prazo persistente e eliminar as alucinações.
> *   É fundamental ter expectativas realistas sobre o que os agentes de IA podem fazer hoje e o que poderão fazer no futuro.
> 
> **5. Aplicações Práticas:**
> 
> *   Agentes de IA podem ajudar a melhorar a eficiência do dia a dia ao automatizar tarefas como organização, atendimento, automação de processos e entretenimento personalizado.
> *   Exemplos de aplicações incluem robo-advisors que ajustam carteiras de investimento e o Browser Operator da Opera, que automatiza tarefas de navegação.
> *   Setores como e-commerce, atendimento ao cliente, marketing e finanças já incorporam agentes de IA.
> 
> **6. Oportunidades de Mercado:**
> 
> *   O mercado de automações e agentes de IA está vivendo uma revolução, com empresas buscando soluções eficientes e escaláveis.
> *   Oportunidades incluem automação de processos internos, personalização de experiências do cliente, análise preditiva e soluções para pequenas e médias empresas.
> *   Criar agentes de IA voltados para nichos específicos pode aumentar a demanda e permitir preços premium.
> 
> Esses são apenas alguns dos lançamentos e tendências mais recentes no mundo dos agentes de IA. À medida que a tecnologia continua a evoluir, podemos esperar ver ainda mais inovações e aplicações surgindo nos próximos meses e anos.
> 


----------------------------------------------------------------

----------------- RESULTADO DO AGENTE PLANEJADOR -----------------



> Para fornecer as últimas notícias sobre lançamentos relevantes de agentes de IA, farei algumas pesquisas no Google. Aqui estão as consultas que usarei:
> 
> Com certeza! Aqui estão alguns dos lançamentos e tendências mais relevantes no mundo dos agentes de IA, com foco nos últimos meses:
> 
> **1. Adoção e Impacto no Comércio:**
> 
> *   Agentes de IA estão se tornando comuns e devem revolucionar a forma como interagimos com as finanças e o comércio. Um relatório da Visa indica que 82% dos brasileiros demonstram interesse em assistentes pessoais com IA.
> *   Empresas de todos os setores estão se adaptando para aproveitar a transformação digital impulsionada pela IA, buscando autonomia, eficiência e redução de custos.
> *   Em 2025, espera-se que a IA deixe de ser uma promessa e se torne uma peça-chave na economia, com empresas investindo fortemente em sistemas mais personalizados e eficientes. Agentes de IA podem ajudar uma empresa com 500 pessoas a trabalhar como se tivesse 1.500 ou 2.000 colaboradores.
> 
> **2. Avanços Tecnológicos e Novas Ferramentas:**
> 
> *   Grandes empresas como IBM, Anthropic, Amazon, Google e OpenAI estão anunciando atualizações e lançamentos que prometem revolucionar o cenário da IA, com ferramentas inovadoras e integração avançada entre agentes.
> *   A IBM anunciou atualizações no watsonx Orchestrate, oferecendo soluções para integrar, customizar e implantar agentes, além de um Agent Catalog que simplifica a orquestração de projetos complexos.
> *   Anthropic introduziu a funcionalidade de busca na web em sua API, permitindo que o agente Claude forneça respostas mais precisas e contextualizadas.
> *   A Microsoft revelou novos agentes para o Security Copilot e proteções para IA. Novos agentes de IA do Copilot criam relatórios e analisam dados em minutos.
> 
> **3. Agentes de IA como Criptomoedas:**
> 
> *   A interseção entre criptomoedas e inteligência artificial está se tornando uma área de crescente interesse, com inovações em IA catalisando a evolução de criptomoedas.
> *   Criptomoedas de IA utilizam inteligência artificial para automatizar tarefas, simplificar operações e oferecer análises preditivas para traders.
> *   Exemplos de criptomoedas de IA incluem NEAR Protocol (NEAR), Fetch.ai (FET) e Ocean Protocol (OCEAN).
> 
> **4. Desafios e Expectativas:**
> 
> *   Especialistas da Andreessen Horowitz (a16z) apontam que o termo "agente de IA" está se desgastando, com muitas pessoas usando a expressão sem entender o conceito.
> *   Para que a IA substitua trabalhadores humanos, é preciso resolver questões técnicas como memória de longo prazo persistente e eliminar as alucinações.
> *   É fundamental ter expectativas realistas sobre o que os agentes de IA podem fazer hoje e o que poderão fazer no futuro.
> 
> **5. Aplicações Práticas:**
> 
> *   Agentes de IA podem ajudar a melhorar a eficiência do dia a dia ao automatizar tarefas como organização, atendimento, automação de processos e entretenimento personalizado.
> *   Exemplos de aplicações incluem robo-advisors que ajustam carteiras de investimento e o Browser Operator da Opera, que automatiza tarefas de navegação.
> *   Setores como e-commerce, atendimento ao cliente, marketing e finanças já incorporam agentes de IA.
> 
> **6. Oportunidades de Mercado:**
> 
> *   O mercado de automações e agentes de IA está vivendo uma revolução, com empresas buscando soluções eficientes e escaláveis.
> *   Oportunidades incluem automação de processos internos, personalização de experiências do cliente, análise preditiva e soluções para pequenas e médias empresas.
> *   Criar agentes de IA voltados para nichos específicos pode aumentar a demanda e permitir preços premium.
> 
> **7. Ferramentas para Criar Agentes de IA:**
> 
> *   Ferramentas No-Code facilitam a criação de agentes de IA para quem não tem conhecimento técnico em programação.
> *   Ferramentas de Processamento de Linguagem Natural (PLN) são essenciais para que os agentes de IA compreendam e reajam à linguagem humana.
> *   Exemplos de ferramentas incluem LangChain, AutoGPT e Microsoft Autogen.
> 
> **8. Tendências:**
> 
> *   Aumento da capacidade dos modelos de IA de lembrar e raciocinar melhor.
> *   Modelos de IA se tornarão mais capazes e úteis, com modelos de raciocínio avançados capazes de resolver problemas complexos.
> *   Agentes de IA ajudarão a simplificar a vida em casa e no trabalho, tornando-se parte integrante de ambos.
> 
> **9. Riscos e Segurança:**
> 
> *   Agentes mal-intencionados usam IA para se passar por autoridades dos EUA e obter acesso a contas de funcionários do governo.
> *   É crucial proteger as ferramentas de IA e usá-las para aumentar a segurança.
> *   A Microsoft está dedicada a ajudar as organizações a proteger seu futuro com uma plataforma de segurança de ponta a ponta que prioriza a IA.
> 
> Esses são apenas alguns dos lançamentos e tendências mais recentes no mundo dos agentes de IA. À medida que a tecnologia continua a evoluir, podemos esperar ver ainda mais inovações e aplicações surgindo nos próximos meses e anos.
> 


----------------------------------------------------------------

----------------- RESULTADO DO AGENTE REDATOR -----------------



> Aqui estão alguns dos lançamentos e tendências mais relevantes no mundo dos agentes de IA nos últimos meses.
> 
> **1. Adoção e Impacto no Comércio:**
> 
> *   Agentes de IA estão se tornando comuns e devem revolucionar a forma como interagimos com as finanças e o comércio. Um relatório da Visa indica que 82% dos brasileiros demonstram interesse em assistentes pessoais com IA.
> *   Empresas de todos os setores estão se adaptando para aproveitar a transformação digital impulsionada pela IA, buscando autonomia, eficiência e redução de custos.
> *   Em 2025, espera-se que a IA deixe de ser uma promessa e se torne uma peça-chave na economia, com empresas investindo fortemente em sistemas mais personalizados e eficientes. Agentes de IA podem ajudar uma empresa com 500 pessoas a trabalhar como se tivesse 1.500 ou 2.000 colaboradores.
> 
> **2. Avanços Tecnológicos e Novas Ferramentas:**
> 
> *   Grandes empresas como IBM, Anthropic, Amazon, Google e OpenAI estão anunciando atualizações e lançamentos que prometem revolucionar o cenário da IA, com ferramentas inovadoras e integração avançada entre agentes.
> *   A IBM anunciou atualizações no watsonx Orchestrate, oferecendo soluções para integrar, customizar e implantar agentes, além de um Agent Catalog que simplifica a orquestração de projetos complexos.
> *   Anthropic introduziu a funcionalidade de busca na web em sua API, permitindo que o agente Claude forneça respostas mais precisas e contextualizadas.
> *   A Microsoft revelou novos agentes para o Security Copilot e proteções para IA. Novos agentes de IA do Copilot criam relatórios e analisam dados em minutos.
> 
> **3. Agentes de IA como Criptomoedas:**
> 
> *   A interseção entre criptomoedas e inteligência artificial está se tornando uma área de crescente interesse, com inovações em IA catalisando a evolução de criptomoedas.
> *   Criptomoedas de IA utilizam inteligência artificial para automatizar tarefas, simplificar operações e oferecer análises preditivas para traders.
> *   Exemplos de criptomoedas de IA incluem NEAR Protocol (NEAR), Fetch.ai (FET) e Ocean Protocol (OCEAN).
> 
> **4. Desafios e Expectativas:**
> 
> *   Especialistas da Andreessen Horowitz (a16z) apontam que o termo "agente de IA" está se desgastando, com muitas pessoas usando a expressão sem entender o conceito.
> *   Para que a IA substitua trabalhadores humanos, é preciso resolver questões técnicas como memória de longo prazo persistente e eliminar as alucinações.
> *   É fundamental ter expectativas realistas sobre o que os agentes de IA podem fazer hoje e o que poderão fazer no futuro.
> 
> **5. Aplicações Práticas:**
> 
> *   Agentes de IA podem ajudar a melhorar a eficiência do dia a dia ao automatizar tarefas como organização, atendimento, automação de processos e entretenimento personalizado.
> *   Exemplos de aplicações incluem robo-advisors que ajustam carteiras de investimento e o Browser Operator da Opera, que automatiza tarefas de navegação.
> *   Setores como e-commerce, atendimento ao cliente, marketing e finanças já incorporam agentes de IA.
> 
> **6. Oportunidades de Mercado:**
> 
> *   O mercado de automações e agentes de IA está vivendo uma revolução, com empresas buscando soluções eficientes e escaláveis.
> *   Oportunidades incluem automação de processos internos, personalização de experiências do cliente, análise preditiva e soluções para pequenas e médias empresas.
> *   Criar agentes de IA voltados para nichos específicos pode aumentar a demanda e permitir preços premium.
> 
> **7. Ferramentas para Criar Agentes de IA:**
> 
> *   Ferramentas No-Code facilitam a criação de agentes de IA para quem não tem conhecimento técnico em programação.
> *   Ferramentas de Processamento de Linguagem Natural (PLN) são essenciais para que os agentes de IA compreendam e reajam à linguagem humana.
> *   Exemplos de ferramentas incluem LangChain, AutoGPT e Microsoft Autogen.
> 
> **8. Tendências:**
> 
> *   Aumento da capacidade dos modelos de IA de lembrar e raciocinar melhor.
> *   Modelos de IA se tornarão mais capazes e úteis, com modelos de raciocínio avançados capazes de resolver problemas complexos.
> *   Agentes de IA ajudarão a simplificar a vida em casa e no trabalho, tornando-se parte integrante de ambos.
> 
> **9. Riscos e Segurança:**
> 
> *   Agentes mal-intencionados usam IA para se passar por autoridades dos EUA e obter acesso a contas de funcionários do governo.
> *   É crucial proteger as ferramentas de IA e usá-las para aumentar a segurança.
> *   A Microsoft está dedicada a ajudar as organizações a proteger seu futuro com uma plataforma de segurança de ponta a ponta que prioriza a IA.
> 
> À medida que a tecnologia continua a evoluir, podemos esperar ver ainda mais inovações e aplicações surgindo nos próximos meses e anos.
> 


----------------------------------------------------------------

----------------- RESULTADO DO AGENTE REVISOR -----------------



> Com certeza! Com base nas informações fornecidas e na data de hoje (16 de maio de 2025), aqui estão alguns dos lançamentos e tendências mais relevantes no mundo dos agentes de IA no último mês:
> 
> 
> Com certeza, aqui estão alguns dos lançamentos e tendências mais relevantes no mundo dos agentes de IA no último mês:
> 
> **Lançamentos e Atualizações Recentes:**
> 
> *   **IBM watsonx Orchestrate:** A IBM anunciou atualizações no watsonx Orchestrate, que oferece soluções para integrar, personalizar e implantar agentes de IA. Inclui um Agent Catalog para simplificar a orquestração de projetos complexos.
> *   **Anthropic:** A Anthropic introduziu a funcionalidade de busca na web em sua API, permitindo que o agente Claude forneça respostas mais precisas e contextualizadas.
> *   **Microsoft Security Copilot:** A Microsoft revelou novos agentes de IA para o Security Copilot e proteções para IA. Os novos agentes de IA do Copilot criam relatórios e analisam dados em minutos.
> *    **GIGABYTE:** A GIGABYTE apresentou sua linha de PCs com IA e desktops projetados para jogos e produtividade, incluindo o agente de IA GiMATE, que permite o controle natural e integrado do notebook por meio da tecnologia LLM.
> *   **Salesforce Agentforce:** A Salesforce está focando seus esforços em agentes de IA e apresentou o Agentforce, uma nova camada na plataforma Salesforce que permite que as empresas criem e implementem agentes de IA que podem agir de forma autônoma em qualquer função de negócios.
> 
> **Ferramentas e Plataformas:**
> 
> *   **Agent.ai:** Uma plataforma inovadora que se posiciona como uma rede profissional para agentes de inteligência artificial. Permite descobrir, conectar-se e contratar agentes de IA para realizar uma variedade de tarefas.
> *   **Ferramentas No-Code:** Ferramentas No-Code facilitam a criação de agentes de IA para quem não tem conhecimento técnico em programação.
> *   **Ferramentas de PLN:** Ferramentas de Processamento de Linguagem Natural (PLN) são essenciais para que os agentes de IA compreendam e reajam à linguagem humana.
> *   **Modelos de Linguagem:** Os modelos de linguagem formam a base do raciocínio do agente, com opções como OpenAI (GPT-4 e variantes), Anthropic (Claude), Mistral AI e modelos open-source como Llama 3.
> 
> **Tendências:**
> 
> *   **Aumento da Capacidade dos Modelos de IA:** Aumento da capacidade dos modelos de IA de lembrar e raciocinar melhor.
> *   **Modelos de IA Mais Capazes:** Modelos de IA se tornarão mais capazes e úteis, com modelos de raciocínio avançados capazes de resolver problemas complexos.
> *   **Integração de Agentes de IA:** Agentes de IA ajudarão a simplificar a vida em casa e no trabalho, tornando-se parte integrante de ambos.
> *   **Expansão do Mercado:** O mercado de agentes de IA está se expandindo rapidamente, com projeções de crescimento de US$ 5 bilhões em 2024 para mais de US$ 47 bilhões em 2030.
> *   **Automação Adaptativa:** Em vez de seguir fluxos pré-definidos, os novos bots usam IA generativa para adaptar suas respostas e decisões em tempo real.
> 
> Espero que esta informação seja útil!
> 


----------------------------------------------------------------
